<a href="https://colab.research.google.com/github/RobAltena/AdventOfCode2025/blob/main/Day10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.optimize import milp, LinearConstraint, Bounds

import heapq
import re
import requests

data = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
"""

data = requests.get('https://raw.githubusercontent.com/RobAltena/AdventofCode2025/refs/heads/main/advent_day10_input.txt').text


lines = data.split('\n')[:-1]

In [ ]:
def pattern_to_integer(pattern: str) -> int:
    return int(pattern.replace('.', '0').replace('#', '1')[::-1], 2) #Put the least significant bit left.


def parse_bit_tuples(str_list: list[str]) -> list[int]:
    """
    Parses a list of strings representing bit tuples and converts them to integers.
    Example: '(1,3)' -> Bits 1 and 3 are ON -> 10 (decimal)
    """
    result_numbers = []

    for item in str_list:
        # 1. Extract all numbers from the string using Regex
        # r'\d+' finds all sequences of digits, ignoring '(', ')', and ','
        bit_indices = [int(n) for n in re.findall(r'\d+', item)]

        # 2. Calculate the integer value
        current_val = 0
        for bit in bit_indices:
            # Shift 1 to the left by 'bit' amount and ADD it using Bitwise OR
            current_val |= (1 << bit)

        result_numbers.append(current_val)

    return result_numbers

In [ ]:
machines = []

for line in lines:
  args = line.split()
  end_state = pattern_to_integer(args[0][1:-1])
  buttons = parse_bit_tuples(args[1:-1])
  end_state_2 = list(map(int, args[-1][1:-1].split(',')))
  machines.append((end_state, buttons, end_state_2))


In [ ]:
def solve_machine(end_state: int, buttons: list[int]) -> int:
  state = 0
  state_set = set()
  state_stack= []
  heapq.heappush(state_stack, (0, 0))

  while state_stack:
    no_presses, state = heapq.heappop(state_stack)
    state_set.add(state)

    no_presses += 1
    new_states = list(map(lambda x: (state ^ x), buttons ))

    if end_state in new_states:
      return no_presses

    new_states = list(filter(lambda x: x not in state_set, new_states))

    for new_state in new_states:
      heapq.heappush(state_stack, (no_presses, new_state))


In [ ]:
# machines = [machines[2]]
result = 0
for end_state, buttons, _ in machines:
  result += solve_machine(end_state, buttons)

print(f"Day 10, part 1: {result}")

In [ ]:
def get_increment_vector(mask: int, n_counters: int) -> np.ndarray:
    bit_indices = np.arange(n_counters)
    return (mask >> bit_indices) & 1

# --- Example Usage ---

# Initialize your counters (e.g., 6 registers)
counters = np.zeros(4, dtype=int)
instruction_mask = 13  # Binary: 001101 (Bits 0, 2, 3 are set)

# Get the array of increments
increments = get_increment_vector(instruction_mask, len(counters))

# Update counters in one step
counters += increments

print(f"Mask:       {instruction_mask} ({bin(instruction_mask)})")
print(f"Increments: {increments}")
print(f"Counters:   {counters}")

In [ ]:
Total_sum = 0
for _, buttons, end_state_2 in machines:

  num_counters = len(end_state_2)
  num_buttons = len(buttons)

  start_state = np.zeros(num_counters, dtype=int)
  Y_list = []
  for button in buttons:
    Y = get_increment_vector(button, num_counters)
    Y_list.append(Y)

  A_original = np.array(Y_list) # Shape (num_buttons, num_counters)

  # For milp, the constraint is A_eq @ x = b_eq
  # Here, x is the vector of button presses (num_buttons variables)
  # A_eq should be (num_counters, num_buttons)
  # b_eq is the target state (num_counters values)
  A_eq_milp = A_original.T # Transpose A to get (num_counters, num_buttons)

  b_eq_milp = np.array(end_state_2)
  c = np.ones(num_buttons) # Cost vector has one entry per button

  constraints = LinearConstraint(A_eq_milp, b_eq_milp, b_eq_milp)
  integrality = np.ones(num_buttons)

  bounds = Bounds(lb=0, ub=np.inf)

  res = milp(c=c, constraints=constraints, integrality=integrality, bounds=bounds)

  if res.success:
      # Rounding is safe because integrality constraint ensures near-integers
      solution = np.round(res.x).astype(int)
      Total_sum += np.sum(solution)
      #print(f"Solution found: x = {solution}")
      #print(f"Total Sum: {np.sum(solution)}")
      # Verification (A_eq_milp @ solution == b_eq_milp)
      #print(f"Check: {A_eq_milp @ solution} == {b_eq_milp}")
  else:
      print("No integer solution exists.")

print(f"Day 10, part 2: {Total_sum}")